In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from itertools import product
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple
from energiapy.model.constraints.constraints import make_constraint, Cons
from energiapy.model.formulate import constraint_export

[    0.00] Initializing mpi-sppy


In [2]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 52  # Number of execution scenarios                     (chi)

M = 1e3  # Big M

In [3]:

def create_list(n_t:int, n:int):
    return [1]*n + [0]*(n_t - n)

def create_event_dict(n_total:int):
    default_list = [1]*n_total
    event_dict = {
    # 'cap1_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1','com1_process'): create_list(n_total, 13)})},
    # 'cap1_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_process'): create_list(n_total, 26)})},
    # 'cap1_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_process'): create_list(n_total, 39)})},
    # 'cap1_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_process'): create_list(52)})},
             
    'cap2_13': {'prob': 0.05, 'factor': pandas.DataFrame(data={('loc2','com1_process'): create_list(n_total, 13)})},
    'cap2_26': {'prob': 0.1, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(n_total, 26)})},
    'cap2_39': {'prob': 0.15, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(n_total, 39)})},
    'cap2_52': {'prob': 0.7, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): default_list})},
     
    # 'cap3_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3','com1_process'): create_list(n_total, 13)})},
    # 'cap3_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_process'): create_list(n_total, 26)})},
    # 'cap3_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_process'): create_list(n_total, 39)})},
    # 'cap3_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_process'): create_list(52)})},
    #  
    # 'cap4_13': {'prob': 0.01, 'factor': pandas.DataFrame(data={('loc4','com1_process'): create_list(n_total, 13)})},
    # 'cap4_26': {'prob': 0.05, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(n_total, 26)})},
    # 'cap4_39': {'prob': 0.1, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(n_total, 39)})},
    # 'cap4_52': {'prob': 0.84, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): default_list})},
    #  
    # 'cap5_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5','com1_process'): create_list(n_total, 13)})},
    # 'cap5_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5', 'com1_process'): create_list(n_total, 26)})},
    # 'cap5_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5', 'com1_process'): create_list(n_total, 39)})},
    # 'cap5_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5', 'com1_process'): create_list(52)})},
    #  
    # 'cap6_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6','com1_process'): create_list(n_total, 13)})},
    # 'cap6_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_process'): create_list(n_total, 26)})},
    # 'cap6_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_process'): create_list(n_total, 39)})},
    # 'cap6_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_process'): create_list(52)})},
    #  
    # 'cap7_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7','com1_process'): create_list(n_total, 13)})},
    # 'cap7_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(n_total, 26)})},
    # 'cap7_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(n_total, 39)})},
    # 'cap7_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(52)})},
    #  
    # 'res1_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1','com1_pur'): create_list(n_total, 13)})},
    # 'res1_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(n_total, 26)})},
    # 'res1_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(n_total, 39)})},
    # 'res1_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(52)})},
    #  
    # 'res6_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6','com1_pur'): create_list(n_total, 13)})},
    # 'res6_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): create_list(n_total, 26)})},
    # 'res6_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): create_list(n_total, 39)})},
    # 'res6_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_pur'): create_list(52)})},
    #  
    # 'trans14_13': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(n_total, 13)})},
    # 'trans14_26': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(n_total, 26)})},
    # 'trans14_39': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(n_total, 39)})},
    # 'trans14_52': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(52)})},
    }
    
    return event_dict

In [4]:
event_dict = create_event_dict(n_total=_exec_scenarios)

In [5]:
# Function to generate the scenario dictionary for n sets of events
def create_scenario_dict(event_dict):
    # Extract unique event prefixes (e.g., 'cap2', 'cap4', ...)
    event_prefixes = set(key.split('_')[0] for key in event_dict)

    # Group events by their prefixes
    grouped_events = {prefix: [key for key in event_dict if key.startswith(prefix)] for prefix in event_prefixes}

    # Create all possible combinations of events across the different groups
    event_combinations = list(product(*grouped_events.values()))

    scenario_dict = {}

    # Iterate over all event combinations
    for combination in event_combinations:
        # Construct the scenario key
        scenario_key = '_'.join(combination)

        # Calculate the probability of this scenario
        prob = 1
        combined_factor = None

        for event_key in combination:
            # Multiply probabilities
            prob *= event_dict[event_key]['prob']

            # Combine factors (assumes they are pandas DataFrames)
            if combined_factor is None:
                combined_factor = event_dict[event_key]['factor'].copy()
            else:
                combined_factor = combined_factor.add(event_dict[event_key]['factor'], fill_value=0)

        # Add to the scenario dictionary
        scenario_dict[scenario_key] = {'prob': prob, 'factor': combined_factor}

    return scenario_dict

# Example usage
scenario_dict = create_scenario_dict(event_dict)

In [6]:
# scenario_dict

In [7]:
# len(scenario_dict)

In [8]:
# sum(scenario_dict[scen]['prob'] for scen in scenario_dict)

In [9]:
# total_prob = sum(scenario_dictionary[name]['prob'] for name in scenario_names)

In [10]:
# for name in scenario_names:
#     scenario_dictionary[name]['prob'] = scenario_dictionary[name]['prob']/total_prob

In [11]:
scenario_names = list(scenario_dict.keys())

In [12]:
# scenario_names

In [13]:
# type(scenario_dict['cap4_13_cap2_26']['factor'])

In [14]:
def build_model(scen_df=pandas.DataFrame()):
    
    default_df = pandas.DataFrame(data=[1]*_exec_scenarios)
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=M, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system', varying=[VaryingResource.DETERMINISTIC_AVAILABILITY])
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
    com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
    com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
    com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
    com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 500
    
    # prod_max = {0: 0.25*com1_process_capacity, 1: 0.5*com1_process_capacity, 2: 0.75*com1_process_capacity, 3: 0.95*com1_process_capacity, 4: com1_process_capacity}
    # prod_min = {0: 0, 1: 0.25*com1_process_capacity, 2: 0.5*com1_process_capacity, 3: 0.75*com1_process_capacity, 4: 0.95*com1_process_capacity}
    # rate_max = {0:1.25/2, 1: 1/2, 2: 0.75/2, 3: 0.5/2, 4: 0.25/2}
    # mode_ramp = {(0,1): 5, (1,2): 5}
    
    com1_procure = Process(name='procure com1', prod_max=com1_process_capacity, conversion={com1_pur: -1, com1_in: 1}, capex=25, vopex=0.01, prod_min=0.01, label='Procure com1')
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Sell com1')
    # com1_opt_procure = Process(name='procure optional com1', prod_max=75, conversion={com1_pur: -1, com1_in:1}, capex=10, vopex=0.1, prod_min=0.01, label='Procure optional com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 1')
    com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 2')
    com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=com1_process_capacity, conversion={com1_loc3_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 3')
    com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=com1_process_capacity, conversion={com1_loc4_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 4')
    com1_receive_loc5 = Process(name='com1_receive_loc5', prod_max=com1_process_capacity, conversion={com1_loc5_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 5')
    com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=com1_process_capacity, conversion={com1_loc6_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 6')
    com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=com1_process_capacity, conversion={com1_loc7_out:-1, com1_in:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 7')
    
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=5, vopex=0.01, prod_min=0.01, label='Process the commodity through the location', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    
    # com1_process = Process(name='com1_process', prod_max=prod_max, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}, 3:{com1_in: -1, com1_out: 1}, 4:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=prod_min, rate_max=rate_max, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    # com1_store10 = Process(name='com1_store10', prod_max=com1_process_capacity, capex=0.1, vopex=0.01, storage_capex=10, store_min=0.01, store_max= 40, prod_min=0.01, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    # com1_store20 = Process(name='com1_store20', prod_max=com1_process_capacity, capex=0.1, vopex=0.02, storage_capex=20, store_min=0.01,store_max= 80, prod_min=0.01, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    # com1_store50 = Process(name='com1_store50', prod_max=com1_process_capacity, capex=0.1, vopex=0.05, storage_capex=50, store_min=0.01, store_max= 200, prod_min=0.01, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    com1_store = Process(name='com1_store', prod_max=500, capex=0.5, vopex=0.01, storage_capex=30, store_min=0.01, store_max=200, prod_min=0.01, label="Storage process", storage=com1_in, storage_cost=0.02)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 1')
    com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 2')
    com1_loc3_send = Process(name='com1_loc3_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc3_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 3')
    com1_loc4_send = Process(name='com1_loc4_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc4_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 4')
    com1_loc5_send = Process(name='com1_loc5_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc5_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 5')
    com1_loc6_send = Process(name='com1_loc6_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc6_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 6')
    com1_loc7_send = Process(name='com1_loc7_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc7_out:1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Send commodity one from location 7')
    
    
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_receive_loc2, com1_receive_loc3, com1_process, com1_store, com1_loc1_send}, label="Location 1",
    scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, 
    availability_factor={com1_pur: scen_df[[('loc1', 'com1_pur')]] if ('loc1', 'com1_pur') in scen_df else default_df})
    
    loc2 = Location(name='loc2', processes={com1_receive_loc1, com1_receive_loc4, com1_receive_loc5, com1_process, com1_store, com1_loc2_send}, label="Location 2", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
    capacity_factor={com1_process: scen_df[[('loc2', 'com1_process')]] if ('loc2', 'com1_process') in scen_df else default_df})
    
    loc3 = Location(name='loc3', processes={com1_receive_loc1, com1_receive_loc4, com1_process, com1_store, com1_loc3_send}, label="Location 3", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc4 = Location(name='loc4', processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc6, com1_receive_loc5, com1_receive_loc7, com1_process, com1_store, com1_loc4_send}, label="Location 4", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
    capacity_factor={com1_process: scen_df[[('loc4', 'com1_process')]] if ('loc4', 'com1_process') in scen_df else default_df})
    
    loc5 = Location(name='loc5', processes={com1_receive_loc2, com1_receive_loc4, com1_receive_loc7, com1_process, com1_store, com1_loc5_send, com1_sell}, label="Location 5", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1)
    
    loc6 = Location(name='loc6', processes={com1_procure, com1_receive_loc4, com1_process, com1_store, com1_loc6_send}, label="Location 6", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
    availability_factor={com1_pur: scen_df[[('loc6', 'com1_pur')]] if ('loc6', 'com1_pur') in scen_df else default_df})
    
    loc7 = Location(name='loc7', processes={com1_receive_loc4, com1_receive_loc5, com1_process, com1_store, com1_loc7_send}, label="Location 7", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
    capacity_factor={com1_process: scen_df[[('loc7', 'com1_process')]] if ('loc7', 'com1_process') in scen_df else default_df})
    
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    truck_cap12 = 4*70
    truck_cap13 = 9*30
    truck_cap24 = 9*50
    truck_cap25 = 9*30
    truck_cap34 = 9*30
    truck_cap45 = 5*100
    truck_cap47 = 9*40
    truck_cap64 = 9*50
    truck_cap75 = 9*40
    
    truck12 = Transport(name='truck12', resources={com1_loc1_out}, trans_max=truck_cap12, label='Truck from location 1 to 2', capex=0.5, vopex=0.05, trans_min=0.01)
    truck21 = Transport(name='truck21', resources={com1_loc2_out}, trans_max=truck_cap12, label='Truck from location 2 to 1', capex=0.0001, vopex=0.05, trans_min=0.01)
    
    truck13 = Transport(name='truck13', resources={com1_loc1_out}, trans_max=truck_cap13, label='Truck from location 1 to 3', capex=0.3, vopex=0.03, trans_min=0.01)
    truck31 = Transport(name='truck31', resources={com1_loc3_out}, trans_max=truck_cap13, label='Truck from location 3 to 1', capex=0.0001, vopex=0.03, trans_min=0.01)
    
    truck24 = Transport(name='truck24', resources={com1_loc2_out}, trans_max=truck_cap24, label='Truck from location 2 to 4', capex=0.5, vopex=0.05, trans_min=0.01)
    truck42 = Transport(name='truck42', resources={com1_loc4_out}, trans_max=truck_cap24, label='Truck from location 4 to 2', capex=0.0001, vopex=0.05, trans_min=0.01)
    
    truck25 = Transport(name='truck25', resources={com1_loc2_out}, trans_max=truck_cap25, label='Truck from location 2 to 5', capex=0.3, vopex=0.03, trans_min=0.01)
    truck52 = Transport(name='truck52', resources={com1_loc5_out}, trans_max=truck_cap25, label='Truck from location 5 to 2', capex=0.0001, vopex=0.03, trans_min=0.01)
    
    truck34 = Transport(name='truck34', resources={com1_loc3_out}, trans_max=truck_cap34, label='Truck from location 3 to 4', capex=0.2, vopex=0.02, trans_min=0.01)
    truck43 = Transport(name='truck43', resources={com1_loc4_out}, trans_max=truck_cap34, label='Truck from location 4 to 3', capex=0.0001, vopex=0.02, trans_min=0.01)
    
    truck45 = Transport(name='truck45', resources={com1_loc4_out}, trans_max=truck_cap45, label='Truck from location 4 to 5', capex=1, vopex=0.1, trans_min=0.01)
    truck54 = Transport(name='truck54', resources={com1_loc5_out}, trans_max=truck_cap45, label='Truck from location 5 to 4', capex=0.0001, vopex=0.1, trans_min=0.01)
    
    truck47 = Transport(name='truck47', resources={com1_loc4_out}, trans_max=truck_cap47, label='Truck from location 4 to 7', capex=0.4, vopex=0.04, trans_min=0.01)
    truck74 = Transport(name='truck74', resources={com1_loc7_out}, trans_max=truck_cap47, label='Truck from location 7 to 4', capex=0.0001, vopex=0.04, trans_min=0.01)
    
    truck64 = Transport(name='truck64', resources={com1_loc6_out}, trans_max=truck_cap64, label='Truck from location 6 to 4', capex=0.5, vopex=0.05, trans_min=0.01)
    truck46 = Transport(name='truck46', resources={com1_loc4_out}, trans_max=truck_cap64, label='Truck from location 4 to 6', capex=0.0001, vopex=0.05, trans_min=0.01)
    
    truck75 = Transport(name='truck75', resources={com1_loc7_out}, trans_max=truck_cap75, label='Truck from location 7 to 5', capex=0.4, vopex=0.04, trans_min=0.01)
    truck57 = Transport(name='truck57', resources={com1_loc5_out}, trans_max=truck_cap75, label='Truck from location 5 to 7', capex=0.0001, vopex=0.04, trans_min=0.01)
    
    
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================
    
    transport_matrix = [
        [[], [truck12], [truck13], [], [], [], []],  # source: location 1
        [[truck21], [], [], [truck24], [truck25], [], []],  # source: location 2
        [[truck31], [], [], [truck34], [], [], []],  # source: location 3
        [[], [truck42], [truck43], [], [truck45], [truck46], [truck47]],  # source: location 4
        [[], [truck52], [], [truck54], [], [], [truck57]],  # source: location 5
        [[], [], [], [truck64], [], [], []],  # source: location 6
        [[], [], [], [truck74], [truck75], [], []]  # source: location 7
    ]
    
    distance_matrix = [
        [0, 55, 196, M, M, M, M],
        [55, 0, M, 163, 112, M, 134],
        [196, M, 0, 63, M, M, M],
        [M, 163, 63, 0, 95, 117, 88],
        [M, 112, M, 95, 0, M, 134],
        [M, M, M, 117, M, 0, M],
        [M, 134, M, 88, 134, M, 0]
    ]
    
    locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7]
    
    sources = locset
    sinks = locset
    
    network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix)



    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 400
    demand_penalty = 20
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc5 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc5 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario', scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Stochastic scenario with Multiple Locations')
    
    if scen_df.empty:
        # ======================================================================================================================
        # Declare problem
        # ======================================================================================================================
        
        problem_mincost = formulate(scenario=scenario,
                                constraints={Constraints.COST, Constraints.TRANSPORT, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                                demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
        

        scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
        capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
        cost_trans_capex = sum(problem_mincost.Capex_transport_network[scale_] for scale_ in scale_iter)

        problem_mincost.first_stage_cost  = capex_process + cost_trans_capex
        
        return scenario, problem_mincost
    
    else:
        return scenario

In [15]:
# scen_test, problem_test = build_model()

In [16]:
# problem_test.constraint_nameplate_production_varying_capacity.pprint()

In [17]:
# scen_test.capacity_factor

In [18]:
# problem_test.del_component('constraint_nameplate_production_varying_capacity')

# Solver Options

In [19]:
solver_options = {
    'MIPGap': 0.005,
    # 'TimeLimit': 60 * 15,
    'Heuristics': 0.20
}

# Perfect Information

In [ ]:
exCost_PI = 0
results_PI = dict()
scen_PI, model_PI = build_model()

In [20]:
for scen_name in scenario_names:
    scen_PI = build_model(scen_df=scenario_dict[scen_name]['factor'])
    
    # Delete process capacity factors, resource availability factors, transport capacity factors
    model_PI.del_component('constraint_nameplate_production_varying_capacity')
    model_PI.del_component('constraint_resource_consumption_varying')
    model_PI.del_component('constraint_export')
    
    # print(scen_PI.capacity_factor)
    # print(scen_PI.availability_factor)
    # print(scen_PI.transport_capacity_factor)
    
    # Add the constraints back for this particular scenario
    model_PI.constraint_nameplate_production_varying_capacity = make_constraint(instance=model_PI, type_cons=Cons.X_LEQ_BY, variable_x='P', location_set=model_PI.locations, component_set=model_PI.processes_varying_capacity,  loc_comp_dict=scen_PI.location_process_dict, b_factor=scen_PI.capacity_factor, x_scale_level=scen_PI.scheduling_scale_level, b_scale_level=scen_PI.capacity_scale_level, y_scale_level=scen_PI.network_scale_level, variable_y='Cap_P', label='restricts production to varying nameplate capacity')
    
    model_PI.constraint_resource_consumption_varying = make_constraint(
                instance=model_PI, type_cons=Cons.X_LEQ_B, variable_x='C', location_set=model_PI.locations, component_set=model_PI.resources_varying_availability, b_max=scen_PI.cons_max,
                loc_comp_dict=scen_PI.location_resource_dict, b_factor=scen_PI.availability_factor, x_scale_level=scen_PI.scheduling_scale_level, b_scale_level=scen_PI.availability_scale_level, label='restricts resource consumption to varying availablity')
    
    constraint_export(instance=model_PI, scheduling_scale_level=scen_PI.scheduling_scale_level,
    network_scale_level=scen_PI.network_scale_level, location_transport_resource_dict=scen_PI.location_transport_resource_dict, transport_capacity_factor=scen_PI.transport_capacity_factor, transport_capacity_scale_level=scen_PI.transport_capacity_scale_level)

    results_PI = solve(scenario=scen_PI, instance=model_PI, solver='gurobi', name=scen_name, solver_options=solver_options)
    
    print('######################## Finished solving '+scen_name+' ########################')
    
    exCost_PI += value(model_PI.objective_cost_w_demand_penalty)*scenario_dict[scen_name]['prob']

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint demand penalty
constraint demand penalty location
constraint deman

In [21]:
exCost_PI

1487130.9500000002

# Expected Value of Perfect Information

In [ ]:
def scenario_creator(scenario_name):  
    scen, model = build_model(scen_df=scenario_dict[scenario_name]['factor'])
    sputils.attach_root_node(model, model.first_stage_cost, [model.X_P, model.Cap_P, model.X_S, model.Cap_S, model.X_F, model.Cap_F])
    model._mpisppy_probability = scenario_dict[scenario_name]['prob']
    return model

In [ ]:
from mpisppy.opt.ef import ExtensiveForm
options = {"solver": "gurobi"}
# all_scenario_names = ["good", "average", "bad"]
ef_UI = ExtensiveForm(options, scenario_names, scenario_creator)
results = ef_UI.solve_extensive_form(solver_options=solver_options)

In [ ]:
exCost_UI = ef_UI.get_objective_value()
print(f"{exCost_UI:.4f}")

In [ ]:
EVPI = exCost_UI - exCost_PI
EVPI

In [ ]:
p_inc = EVPI*100/exCost_PI
p_inc

In [ ]:
ssoln_dict = ef_UI.get_root_solution()

# Value of Stochastic Solution

In [ ]:
# n_avg = int(sum(ns_dict[i] for i in ns_dict.keys())/3)
# cap_factor_avg = pandas.DataFrame(data={'com1_process': [1]*n_avg + [0]*(_exec_scenarios-n_avg)})
# scen_avg, model_avg = build_model(cap_factor=cap_factor_avg)
# 
# solver_avg = SolverFactory("gurobi")
# solver_avg.solve(model_avg, options=solver_options)

In [ ]:
# def fix_variables(model1: ConcreteModel, model2:ConcreteModel):
#     
#     def fix(var1, var2):
#         for i in list(var1.keys()):
#             if var1[i].value is None:
#                 continue
#             else:
#                 var2[i].fixed = True
#                 var2[i] = value(var1[i])
#             # var2[i].pprint()
#             
#     fix(model1.X_P, model2.X_P)
#     fix(model1.Cap_P, model2.Cap_P)
#     fix(model1.X_S, model2.X_S)
#     fix(model1.Cap_S, model2.Cap_S)
#     fix(model1.X_F, model2.X_F)
#     fix(model1.Cap_F, model2.Cap_F)

In [ ]:
# exCost_FD = 0
# 
# for scenario_name in all_scenario_names:
#     cap_factor_VSS = pandas.DataFrame(data={'com1_process': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
#     scen_VSS, model_VSS = build_model(cap_factor=cap_factor_VSS)
#     
#     fix_variables(model1=model_avg, model2=model_VSS)
#     
#     solver_VSS = SolverFactory("gurobi")
#     solver_VSS.solve(model_VSS, options=solver_options)
#     
#     print('######################## Finished solving '+scenario_name+' ########################')
#     
#     exCost_FD += value(model_VSS.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

In [ ]:
# VSS = exCost_FD - exCost_UI
# VSS

# Results Summary

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

# Define nodes (suppliers, factories, warehouses, etc.) with additional labels
nodes = {
    'Location 1': {
        'Capacity': ssoln_dict['Cap_P[loc1,com1_process,0]'] if 'Cap_P[loc1,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc1,com1_store,0]'] if 'Cap_P[loc1, com1_store,0]' in ssoln_dict.keys() else 0
    },
    'Location 2': {
        'Capacity': ssoln_dict['Cap_P[loc2,com1_process,0]'] if 'Cap_P[loc2,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc2,com1_store,0]'] if 'Cap_P[loc2, com1_store,0]' in ssoln_dict.keys() else 0
    },
    'Location 3': {
        'Capacity': ssoln_dict['Cap_P[loc3,com1_process,0]'] if 'Cap_P[loc3,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc3,com1_store,0]'] if 'Cap_P[loc3, com1_store,0]' in ssoln_dict.keys() else 0
    },
    'Location 4': {
        'Capacity': ssoln_dict['Cap_P[loc4,com1_process,0]'] if 'Cap_P[loc4,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc4,com1_store,0]'] if 'Cap_P[loc4, com1_store,0]' in ssoln_dict.keys() else 0
    },
    'Location 5': {
        'Capacity': ssoln_dict['Cap_P[loc5,com1_process,0]'] if 'Cap_P[loc5,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc5,com1_store,0]'] if 'Cap_P[loc5, com1_store,0]' in ssoln_dict.keys() else 0
    },
    'Location 6': {
        'Capacity': ssoln_dict['Cap_P[loc6,com1_process,0]'] if 'Cap_P[loc6,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc6,com1_store,0]'] if 'Cap_P[loc6, com1_store,0]' in ssoln_dict.keys() else 0
    },
    'Location 7': {
        'Capacity': ssoln_dict['Cap_P[loc7,com1_process,0]'] if 'Cap_P[loc7,com1_process,0]' in ssoln_dict.keys() else 0,
        'Inventory': ssoln_dict['Cap_P[loc7,com1_store,0]'] if 'Cap_P[loc7, com1_store,0]' in ssoln_dict.keys() else 0
    },
}

# Add nodes to the graph
G.add_nodes_from(nodes.keys())

# Define edges (connections between nodes)
edges = [
    ('Location 1', 'Location 2', {'weight': ssoln_dict['Cap_F[loc1,loc2,truck12,0]'] if 'Cap_F[loc1,loc2,truck12,0]' in ssoln_dict.keys() else 0}),
    ('Location 1', 'Location 3', {'weight': ssoln_dict['Cap_F[loc1,loc3,truck13,0]'] if 'Cap_F[loc1,loc3,truck13,0]' in ssoln_dict.keys() else 0}),
    ('Location 2', 'Location 4', {'weight': ssoln_dict['Cap_F[loc2,loc4,truck24,0]'] if 'Cap_F[loc2,loc4,truck24,0]' in ssoln_dict.keys() else 0}),
    ('Location 2', 'Location 5', {'weight': ssoln_dict['Cap_F[loc2,loc5,truck25,0]'] if 'Cap_F[loc2,loc5,truck25,0]' in ssoln_dict.keys() else 0}),
    ('Location 3', 'Location 4', {'weight': ssoln_dict['Cap_F[loc3,loc4,truck34,0]'] if 'Cap_F[loc3,loc4,truck34,0]' in ssoln_dict.keys() else 0}),
    ('Location 4', 'Location 5', {'weight': ssoln_dict['Cap_F[loc4,loc5,truck45,0]'] if 'Cap_F[loc4,loc5,truck45,0]' in ssoln_dict.keys() else 0}),
    ('Location 4', 'Location 7', {'weight': ssoln_dict['Cap_F[loc4,loc7,truck47,0]'] if 'Cap_F[loc4,loc7,truck47,0]' in ssoln_dict.keys() else 0}),
    ('Location 6', 'Location 4', {'weight': ssoln_dict['Cap_F[loc6,loc4,truck64,0]'] if 'Cap_F[loc6,loc4,truck64,0]' in ssoln_dict.keys() else 0}),
    ('Location 7', 'Location 5', {'weight': ssoln_dict['Cap_F[loc7,loc5,truck75,0]'] if 'Cap_F[loc7,loc5,truck75,0]' in ssoln_dict.keys() else 0}),
]

# Add edges to the graph
G.add_edges_from(edges)

# Define fixed positions for nodes
fixed_positions = {
    'Location 1': (-2, 2),
    'Location 2': (0, 2),
    'Location 3': (-2, 0),
    'Location 4': (0, 0),
    'Location 5': (2, 0),
    'Location 6': (-2, -2),
    'Location 7': (1, -2)
}

# Create custom labels combining multiple pieces of information
node_labels = {node: f"Capacity: {data['Capacity']}\nInventory: {data['Inventory']}" for node, data in nodes.items()}

# Draw nodes and edges with fixed positions
nx.draw_networkx_nodes(G, fixed_positions, node_color='skyblue', node_size=700)
nx.draw_networkx_edges(G, fixed_positions, edgelist=edges, arrowstyle='simple', arrowsize=20)
nx.draw_networkx_labels(G, fixed_positions, labels=node_labels, font_size=8, font_color='green', font_weight='bold')

# Draw edge labels (optional)
edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, fixed_positions, edge_labels=edge_labels)

# Show the plot
plt.title("Supply Chain Network with Fixed Node Positions")
plt.show()


In [ ]:
EVPI

In [ ]:
p_inc

In [ ]:
# import matplotlib.pyplot as plt
# import networkx as nx
# 
# # Create a directed graph
# G = nx.DiGraph()
# 
# # Define nodes with additional labels
# nodes = {
#     'Supplier1': {'type': 'Supplier', 'inventory': 100},
#     'Supplier2': {'type': 'Supplier', 'inventory': 150},
#     'Factory': {'type': 'Factory', 'inventory': 200},
#     'Warehouse': {'type': 'Warehouse', 'inventory': 300},
#     'Customer': {'type': 'Customer', 'inventory': 0}
# }
# 
# # Add nodes and edges to the graph
# G.add_nodes_from(nodes.keys())
# edges = [
#     ('Supplier1', 'Factory', {'weight': 10}),
#     ('Supplier2', 'Factory', {'weight': 15}),
#     ('Factory', 'Warehouse', {'weight': 20}),
#     ('Warehouse', 'Customer', {'weight': 25}),
# ]
# G.add_edges_from(edges)
# 
# # Define fixed positions for nodes
# fixed_positions = {
#     'Supplier1': (-1, 1),
#     'Supplier2': (-1, -1),
#     'Factory': (0, 0),
#     'Warehouse': (1, 0),
#     'Customer': (2, 0)
# }
# 
# # Create custom labels combining multiple pieces of information
# node_labels = {node: f"{data['type']}\nInventory: {data['inventory']}" for node, data in nodes.items()}
# 
# # Draw nodes and edges
# nx.draw_networkx_nodes(G, fixed_positions, node_color='skyblue', node_size=1500)
# nx.draw_networkx_edges(G, fixed_positions, edgelist=edges, arrowstyle='->', arrowsize=20)
# 
# # Draw node labels next to the nodes
# for node, (x, y) in fixed_positions.items():
#     label = node_labels[node]
#     plt.text(x + 0.1, y, label, fontsize=10, fontweight='bold', ha='left', va='center')
# 
# # Draw edge labels (optional)
# edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G, fixed_positions, edge_labels=edge_labels)
# 
# # Show the plot
# plt.title("Supply Chain Network with Labels Next to Nodes")
# plt.show()
